# Đồ án cuối kì - Thực hành Nhập môn khoa học dữ liệu
### Nhóm: 16


## I. THU THẬP DỮ LIỆU

**Thêm các thư viện cần thiết**

In [ ]:
from selenium import webdriver
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd

**Các trường dữ liệu bệnh sẽ lấy**

Để dễ dàng hơn cho việc thu thập dữ liệu, chúng em tổng hợp sẵn những link cần lấy dữ liệu đưa vào dictionary.

Chúng em chọn ra những nguyên nhân sau:
- HIV/AIDS
- Malaria
- Tuberculosis
- Dengue
- Covid-19
- Brain and nervous system cancers
- Leukaemia
- Road traffic accidents
- Fires
- Drownings
- Natural disasters
- Self-inflicted injuries
- Violence
- War

In [ ]:
cause_of_death_url = {
    'HIV/AIDS': 'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/hiv-aids',
    'Malaria': 'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/malaria',
    'Tuberculosis': 'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/tuberculosis',
    'Dengue': 'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/dengue',
    'Covid-19': 'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/covid-19',
    'Brain and nervous system cancers': 'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/brain-and-nervous-system-cancers',
    'Leukaemia': 'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/leukaemia',
    'Road traffic accidents': 'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/road-traffic-accidents',
    'Fires': 'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/fires',
    'Drownings': 'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/drownings',
    'Natural disasters': 'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/natural-disasters',
    'Self-inflicted injuries': 'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/self-inflicted-injuries',
    'Violence': 'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/violence',
    'War': 'https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/war'
}

**Tạo file csv để lưu dữ liệu**

In [ ]:
# tạo trước file data.csv, data của mỗi cause sẽ được lưu lần lượt vào file này
pd.DataFrame({
    "Cause of death": [],
    "Age": [],
    "Year": [],
    "Country": [],
    "Number of deaths": []
}).to_csv(r'data.csv', index = None, header=True)

**Crawl dữ liệu từ web**

In [ ]:
for cause in cause_of_death_url:
    # cài đặt và khởi động Chrome
    driver = webdriver.Chrome(ChromeDriverManager().install())
    
    print(cause)

    cause_of_death, age, year, country, number_of_deaths = [], [], [], [], []

    # mở page
    driver.get(cause_of_death_url[cause])
    
    # click để chọn biểu đồ
    driver.find_element(By.ID, 'visualization_component_4_key').click()
    sleep(3)
    
    # click để bỏ chọn năm 2021
    driver.find_element(By.ID, '2021').click()
    
    # chọn all quốc gia
    driver.find_element(By.XPATH, "/html/body/div[2]/section/div[3]/div/div[2]/div/div[2]/div[2]/div/div[3]/div/div[2]/div[1]/div[3]/div/div/input").click()
    driver.find_element(By.XPATH, "/html/body/div[2]/section/div[3]/div/div[2]/div/div[2]/div[2]/div/div[3]/div/div[2]/div[1]/div[3]/div/div/div[2]/ul/li[1]/label/input").click()
    
    # chọn giới tính (ALL)
    # driver.find_element(By.ID, 'filter_indicatorlinechart_countries5_key').click()
    # driver.find_element(By.XPATH, "/html/body/div[2]/section/div[3]/div/div[2]/div/div[2]/div[2]/div/div[3]/div/div[2]/div[1]/div[5]/div/div/select/option[1]").click()
    
    # tìm nhóm tuổi
    driver.find_element(By.ID, 'filter_indicatorlinechart_countries8_key').click()
    ages_html = driver.find_elements(By.XPATH, '/html/body/div[2]/section/div[3]/div/div[2]/div/div[2]/div[2]/div/div[3]/div/div[2]/div[1]/div[4]/div/div/select/option')

    for age_html in ages_html[1:-1]:
        # click để chọn nhóm tuổi
        age_html.click()
    
        # chọn áp dụng bộ lọc
        driver.find_element(By.ID, 'filters_section_indicatorlinechart_countries_key_filter_button').click()
        # chờ để loading page
        sleep(10)
    
        # lấy data
        # ------------------------
        while(True):
            table_html = driver.find_element(By.ID, 'custom_table')
            # lấy tên cột
            cols_name_html = table_html.find_elements(By.TAG_NAME, "th")
            cols_name = [col.text for col in cols_name_html[1:]]
            # lấy dòng
            rows_html = table_html.find_elements(By.XPATH, "/html/body/div[2]/section/div[3]/div/div[2]/div/div[2]/div[2]/div/div[3]/div/div[2]/div[2]/div[3]/div[1]/table/tbody/tr")
    
            for row_html in rows_html:
                values_html = row_html.find_elements(By.TAG_NAME, "td")
                for value_html in values_html[1:]:
                    number_of_deaths.append(value_html.text)
                year += [values_html[0].text] * len(cols_name)
            
            country += cols_name * len(rows_html)
            age += [age_html.text] * len(rows_html) * len(cols_name)
            cause_of_death += [cause] * len(rows_html) * len(cols_name)
    
            try:
                # click để qua trang
                driver.find_element(By.ID, 'next_button').click()
            except:
                break
        # ------------------------
    
    # lưu data vừa lấy vào file data.csv
    df = pd.DataFrame({
        "Cause of death": cause_of_death,
        "Age": age,
        "Year": year,
        "Country": country,
        "Number of deaths": number_of_deaths 
    })
    df.to_csv(r'data.csv', index = None, header = False, mode = 'a')

    # đóng Chrome
    driver.quit()